Script to post-process high-resolution WRF model output.

Major tasks include computing the following for selected variables:
  1. domain-averages to produce time series
  2. vertical integrals
  3. pressure-level vertical interpolation

James Ruppert  
25 Nov 2024

In [12]:
from netCDF4 import Dataset
import numpy as np
from wrf import getvar, ALL_TIMES#, vinterp
import xarray as xr
from read_wrf_piccolo import *
from post_proc_functions import *
import os

In [13]:
do_2d_special = False
do_3d_vars = False
do_3d_special = False
do_rainrate = True # Set select=1:ncpus=1:mpiprocs=1:ompthreads=1

In [14]:
########################################################
# Directories and test selection
########################################################

datdir = "/glade/derecho/scratch/ruppert/piccolo/"
# datdir = "/glade/campaign/univ/uokl0053/"
# datdir = "/ourdisk/hpc/radclouds/auto_archive_notyet/tape_2copies/piccolo/"

case = "sept1-4"
test_process = "ncrf12h"
wrf_dom = "wrf_fine"
nmem = 5 # number of ensemble members

########################################################
# New functions and single-use calls go here
########################################################

# Ens-member string tags (e.g., memb_01, memb_02, etc.)
memb0=1 # Starting member to read
memb_nums_str=np.arange(memb0,nmem+memb0,1).astype(str)
nustr = np.char.zfill(memb_nums_str, 2)
memb_all=np.char.add('memb_',nustr)

In [15]:
def calculate_rainrate(rainnc_all, npd):
    rainrate = rainnc_all.copy()
    nt_all = rainrate.shape[0]
    rainrate[0] = 0
    rainrate[nt_all-1] = np.nan
    rainrate[1:-1] = (rainnc_all.values[2:] - rainnc_all.values[:-2])*0.5
    rainrate *= npd # Convert to mm/day
    return rainrate

In [16]:
if do_rainrate:

        # for memb_dir in memb_all:
        memb_dir = memb_all[0]

        outdir, wrffiles, nfiles, npd = memb_dir_settings(datdir, case, test_process, wrf_dom, memb_dir)

        ds = xr.open_dataset(outdir+'RAINNC.nc')
        rainnc = ds['RAINNC']
        # Get rainrate
        rainrate = calculate_rainrate(rainnc, npd)
        # rainrate = xr.DataArray(rainrate, coords=rainnc.coords, dims=rainnc.dims, attrs=rainnc.attrs)
        # Write out
        var_name='rainrate'
        write_ncfile(outdir, rainrate, var_name)

        print("Done writing out rainrate")

Done writing out rainrate


In [18]:
npd

72